## Required Libraries

In [16]:
import re
import emoji
import pandas
import numpy
import csv
import contractions
import fasttext

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")

## Dataset

In [3]:
tweet_data = pandas.read_csv("tweet_data.csv")

In [14]:
tweet_data.head(5)

,sentiment,processed_text
0,__label__negative,layin n bed with a headache ughh.waitin on yo...
1,__label__negative,funeral ceremony.gloomy friday.
2,__label__positive,wants to hang out with friends soon!
3,__label__negative,re-pinging : why did not you go to prom? becau...
4,__label__negative,hmm. is down


In [5]:
# Drop NAs
tweet_data = tweet_data.dropna()

## Preprocessing

In [6]:
# Handling the retweet(RT) tag
def replace_retweet(tweet, default_replace=""):
    tweet = re.sub('RT\s+', default_replace, tweet)
    return tweet

# Handling user tags(@)
def replace_user(tweet, default_replace="user"):
    tweet = re.sub('\B@\w+', default_replace, tweet)
    return tweet

# Replace emojis with meaningful text
def demojize(tweet):
    tweet = emoji.demojize(tweet)
    return tweet

def replace_url(tweet, default_replace=""):
    tweet = re.sub('(http|https):\/\/\S+', default_replace, tweet)
    return tweet

# Remove hashtag symbol(#)
def replace_hashtag(tweet, default_replace=""):
    tweet = re.sub('#+', default_replace, tweet)
    return tweet

def to_lowercase(tweet):
  tweet = tweet.lower()
  return tweet

def word_repetition(tweet):
  tweet = re.sub(r'(.)\1+', r'\1\1', tweet)
  return tweet

def punct_repetition(tweet, default_replace=""):
  tweet = re.sub(r'[\?\.\!]+(?=[\?\.\!])', default_replace, tweet)
  return tweet

# Replace contractions with their extended forms
def fix_contractions(tweet):
  tweet = contractions.fix(tweet)
  return tweet

In [7]:
def preprocessor(tweet, verbose=False):
    if verbose: print("Initial tweet: {}".format(tweet))

    ## Twitter Features
    tweet = replace_retweet(tweet) # replace retweet
    tweet = replace_user(tweet, "") # replace user tag
    tweet = replace_url(tweet) # replace url
    tweet = replace_hashtag(tweet) # replace hashtag
    if verbose: print("Post Twitter processing tweet: {}".format(tweet))

    ## Word Features
    tweet = to_lowercase(tweet) # lower case
    tweet = fix_contractions(tweet) # replace contractions
    tweet = punct_repetition(tweet) # replace punctuation repetition
    tweet = word_repetition(tweet) # replace word repetition
    tweet = demojize(tweet) # replace emojis
    if verbose: print("Post Word processing tweet: {}".format(tweet))

    return tweet

In [8]:
tweet_data['processed_text'] = tweet_data['tweet_text'].apply(preprocessor)

In [10]:
# Drop extra columns
tweet_data = tweet_data.drop(['textID', 'tweet_text'], axis=1)

In [11]:
# Add __label__ tags
tweet_data['sentiment']=['__label__'+s.replace(' or ', '$').replace(', or ','$').replace(',','$').replace(' ','_').replace(',','__label__').replace('$$','$').replace('$',' __label__').replace('___','__') for s in tweet_data['sentiment']]
tweet_data['processed_text']= tweet_data['processed_text'].replace('\n',' ', regex=True).replace('\t',' ', regex=True)

In [12]:
tweet_data.head(5)

,sentiment,processed_text
0,__label__negative,layin n bed with a headache ughh.waitin on yo...
1,__label__negative,funeral ceremony.gloomy friday.
2,__label__positive,wants to hang out with friends soon!
3,__label__negative,re-pinging : why did not you go to prom? becau...
4,__label__negative,hmm. is down


## Split dataset

Train dataset size = 18727*75% = 14045

Test dataset size = 18727-14045 = 4682

In [17]:
tweet_data.head(14045).to_csv('tweet_data_fasttext.train.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ", encoding = 'utf-8')
tweet_data.tail(4682).to_csv('tweet_data_fasttext.test.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ", encoding = 'utf-8')

## Model training and prediction

In [18]:
# Train models using fastText and autotune hyperparameters
model = fasttext.train_supervised('tweet_data_fasttext.train.txt')

In [19]:
prediction = model.test_label('tweet_data_fasttext.test.txt')

## Model evaluation

In [20]:
def calculate_accuracy(model):
    #Getting data from a CSV
    labels = tweet_data['sentiment']
    tweets = tweet_data['processed_text']
    
    total, correct = 0, 0
    
    for idx, value in tweets.iteritems():
        #Predicting
        prediction = model.predict(value, k=1)
        
        #Saving accuracy
        total += 1
        if(prediction[0][0] == labels[idx]):
            correct += 1
            
    accuracy = correct / total
    print('accuracy = ' + 'correct/total = '+ str(correct) + '/' + str(total) + ' = ' + str(accuracy) + '\n')

In [23]:
def evaluate_model(dict):
    label_0 = dict['__label__negative']
    label_1 = dict['__label__positive']
    
    p0 = label_0['precision']
    r0 = label_0['recall']
    f0 = label_0['f1score']
    p1 = label_1['precision']
    r1 = label_1['recall']
    f1 = label_1['f1score']
    
    macro_precision = (p0 + p1) / 2
    macro_recall = (r0 + r1) / 2
    macro_f1 = (f0+ f1) / 2
    
    print('macro_precision = ' + str(macro_precision))
    print('macro_recall = ' + str(macro_recall))
    print('macro_f1 = ' + str(macro_f1))

In [24]:
evaluate_model(prediction)
calculate_accuracy(model)

macro_precision = 0.7085238162178262
macro_recall = 0.828588096992666
macro_f1 = 0.7442228899207867
accuracy = correct/total = 17333/18727 = 0.9255620227479041

